# Custom spider v0.0
Very frst generation of custom spider - base for Custom framework, tested on allrecipes.com

In [22]:
import urllib.robotparser
import urllib.request

import sys
import time
import json

from bs4 import BeautifulSoup

In [23]:
# allrecipes get directions
def directions_allrecipes(soup):
    temp = list(map(lambda x: x.get_text().strip(), soup.find_all('span', 'recipe-directions__list--item')))
    temp = list(filter(lambda x: x != '', temp))
    return temp

In [24]:
# allrecipes get ingredients
def ingredients_allrecipes(soup):
    temp = list(map(lambda x: x.get_text().strip(), soup.find_all('span', 'recipe-ingred_txt')))
    temp = list(filter(lambda x: x != '' and x != 'Add all ingredients to list', temp))
    return temp

In [25]:
# allrecipes get title
def title_allrecipes(soup):
    return list(map(lambda x: x.get_text().strip(), soup.find_all('h1','recipe-summary__h1')))[0]

In [26]:
def recipe_allrecipes(soup):
    ret = dict()
    ret['title'] = title_allrecipes(soup)
    ret['ingredients'] = ingredients_allrecipes(soup)
    ret['directions'] = directions_allrecipes(soup)
    return ret

In [27]:
def handle_recipe(soup, retrive_recipe, retrive_title, path="."):
    title = retrive_title(soup)
    body = json.dumps(retrive_recipe(soup))
    # TODO check if such a file exists
    with open(path+'/'+title.lower().replace(' ','_')+'.json', 'w+') as f:
        f.write(body)

## test

In [10]:
# get test page
html_doc = urllib.request.urlopen('https://www.allrecipes.com/recipe/192256/bayou-crostini/').read().decode('utf-8')
soup = BeautifulSoup(html_doc, 'html.parser')

In [16]:
recipe_allrecipes(soup)

{'title': 'Bayou Crostini',
 'ingredients': ['1 whole grain baguette',
  '2 tablespoons olive oil',
  '3/4 pound Brie cheese',
  '24 oysters - shucked, rinsed and drained',
  '1/4 cup butter, melted',
  'salt and pepper to taste',
  '1 red bell pepper, cut into thin strips'],
 'directions': ['Preheat oven to 350 degrees F (175 degrees C). Slice the baguette into 24 rounds. Brush bread rounds lightly with olive oil. Place bread rounds on a baking sheet.',
  'Toast the bread rounds in the preheated oven just until crisp, 5 to 8 minutes. Meanwhile, slice Brie cheese into 24 equal pieces. Remove toasted bread from oven; top each round with a slice of Brie cheese. Return the crostini to the oven to melt the cheese, about 5 minutes.',
  "Preheat the oven's broiler and set the oven rack about 6 inches from the heat source. In an ovenproof casserole, stir together the oysters, butter, any remaining olive oil, salt, and pepper.",
  'Broil in the preheated oven until the oysters begin to curl at

In [19]:
handle_recipe(soup, recipe_allrecipes, title_allrecipes)

# Full crawler
urllib + beautiful soup - retrives list of links to recipes (does not download recipes - yet)

In [28]:
musthave = 'https://www.allrecipes.com/recipe/'
seed = 'https://www.allrecipes.com/'
avoid = [
    '/photos/',
    '/reviews/',
    '/video/'
]

In [29]:
def is_valid(url, visited = [], musthave = musthave, seed = seed, avoid = avoid):
    if url in visited:        # if url was not visited before
        return False
    if url.find(seed) != 0:   # if it is in domain
        return False
    for a in avoid:           # if it is not to be avoided (photos, reviews, etc)
        if url.find(a) >= 0:
            return False
    return True

In [8]:
# load allrecipes hubs
with open('allrecipes/hubs.txt') as f:
    hubs = f.readlines()

hubs = list(map(lambda x: x.strip(), hubs))

'https://www.allrecipes.com/'

In [8]:
links = []
links.append(seed)
visited = set()
recipes = set()
counter = 1
total = 0

# add hubs to links
links.extend(hubs)

while links:
    current = links.pop(0)
    try:
        start = time.time()
        # get page
        html_doc = urllib.request.urlopen(current).read().decode('utf-8')
        visited.add(current)
        # retirive urls from page
        soup = BeautifulSoup(html_doc, 'html.parser')
        tlinks = list(map(lambda x: x['href'], soup.find_all('a', href=True)))
        
        for l in tlinks:
            #if l not in visited and l.find(seed) == 0: #valid
            if is_valid(l, visited = visited):
                if l.find(musthave) == 0: #is a recipe
                    recipes.add(l)
                links.append(l)
        links = list(set(links)) # remove duplicates
        
        end = time.time()
        elapsed = end-start
        total += elapsed
        
        print("Step:\t", counter, "\tRecipes:\t", len(recipes), "\tLinks:\t", len(links), "\tVisited:\t", len(visited), "\tTime:\t", elapsed, "\tMean time:\t", total/counter)
        time.sleep(max(0, 1.1-elapsed))
        
    except KeyboardInterrupt:
        print("Finish and save")
        break
    except:
        print('Error while parsing: ', current)
        print(sys.exc_info()[0])
    # counter control
    counter += 1
    if counter > 10: # presentation mode
        break
    if counter % 200 == 0: #save temporary results
        with open('tmp.txt', 'w+') as f:
            for r in recipes:
                f.write(r+'\n')

Step:	 1 	Recipes:	 20 	Links:	 91 	Visited:	 1 	Time:	 6.839080333709717 	Mean time:	 6.839080333709717
Step:	 2 	Recipes:	 20 	Links:	 96 	Visited:	 2 	Time:	 4.123126745223999 	Mean time:	 5.481103539466858
Step:	 3 	Recipes:	 24 	Links:	 110 	Visited:	 3 	Time:	 5.3405985832214355 	Mean time:	 5.434268554051717
Finish and save


# Retrieved links crawler

In [1]:
%%bash 
ls allrecipes/recipes | wc -l

5833


In [21]:
# read links
with open('allrecipes/links.txt') as f:
    links = f.readlines()

links = list(map(lambda x: x.strip(), links))

In [39]:
total = 0
counter = 1

for l in links[5000:6000]:
    try:
        start = time.time()
        
        html_doc = urllib.request.urlopen(l).read().decode('utf-8')
        soup = BeautifulSoup(html_doc, 'html.parser')
        
        handle_recipe(soup, retrive_recipe=recipe_allrecipes, retrive_title=title_allrecipes, path='allrecipes/recipes')
        
        end = time.time()
        elapsed = end-start
        total += elapsed
        
        print("Step:\t", counter, "\tTime:\t", elapsed, "\tMean time:\t", total/counter)
        time.sleep(max(0, 1.1-elapsed))
        
        counter += 1
        
    except KeyboardInterrupt:
        print("Finish and save")
        break
    except:
        print("Error while parsing: ", l)

Step:	 1 	Time:	 3.4965295791625977 	Mean time:	 3.4965295791625977
Step:	 2 	Time:	 3.842465877532959 	Mean time:	 3.6694977283477783
Step:	 3 	Time:	 3.629589796066284 	Mean time:	 3.656195084253947
Step:	 4 	Time:	 3.4229581356048584 	Mean time:	 3.597885847091675
Step:	 5 	Time:	 3.8070411682128906 	Mean time:	 3.639716911315918
Step:	 6 	Time:	 3.7760062217712402 	Mean time:	 3.662431796391805
Step:	 7 	Time:	 4.118542432785034 	Mean time:	 3.7275904587336948
Step:	 8 	Time:	 3.770151138305664 	Mean time:	 3.732910543680191
Step:	 9 	Time:	 3.555030107498169 	Mean time:	 3.7131460507710776
Step:	 10 	Time:	 3.0694494247436523 	Mean time:	 3.648776388168335
Step:	 11 	Time:	 3.805778741836548 	Mean time:	 3.6630493294109
Step:	 12 	Time:	 3.6316325664520264 	Mean time:	 3.6604312658309937
Step:	 13 	Time:	 3.6462154388427734 	Mean time:	 3.659337740678054
Step:	 14 	Time:	 8.21797490119934 	Mean time:	 3.9849546807152882
Step:	 15 	Time:	 3.385371208190918 	Mean time:	 3.9449824492